# Ottawa Traffic Accident Data Analytic- Data Cleaning

In [1]:
#import basic libraries
import pandas as pd
import numpy as np
import geopandas as gpd # geojson library
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim#use geopy library to get 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json
from shapely.ops import nearest_points
from datetime import datetime

## Import traffic dataset 

In [3]:
### file path of ottawa traffic accident data 2014-2017 
collision_2013 = 'C:/Users/project file/collisionsOttawa2013.csv'
collision_2014 = 'C:/Users/project file/collisionsOttawa2014.csv'
collision_2015 = 'C:/Users/project file/collisionsOttawa2015.csv'
collision_2016 = 'C:/Users/project file/collisionsOttawa2016.csv'
collision_2017 = 'C:/Users/project file/collisionsOttawa2017.csv'
# read csv to dataframe
df_2013 = pd.read_csv(collision_2013)
df_2014 = pd.read_csv(collision_2014)
df_2015 = pd.read_csv(collision_2015)
df_2016 = pd.read_csv(collision_2016)
df_2017 = pd.read_csv(collision_2017)

In [4]:
# Make Consistency of data columns in each dataset
df_2017 = df_2017.rename(columns={'Record':'COLLISION_ID'})
df_2017 = df_2017.drop(columns=['Year','Collision_Location'])

# Adjust the order of the data columns in the dataset 2017
df_2017 = df_2017[['COLLISION_ID', 'LOCATION', 'X', 'Y', 'LONGITUDE', 'LATITUDE', 'DATE',
       'TIME', 'ENVIRONMENT', 'LIGHT', 'SURFACE_CONDITION', 'TRAFFIC_CONTROL', 'COLLISION_CLASSIFICATION', 'IMPACT_TYPE']]

# concatenating df_2015,df_2016,df_2014 axix = 0
df_concatenated = pd.concat([df_2014,df_2015,df_2016], ignore_index=True)
df_concatenated = df_concatenated.drop(columns=['TRAFFIC_CONTROL_CONDITION','NO_OF_PEDESTRIANS'])
df_combine = pd.concat([df_concatenated,df_2017],ignore_index=True)

df_combine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58345 entries, 0 to 58344
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   COLLISION_ID              58345 non-null  object 
 1   LOCATION                  58345 non-null  object 
 2   X                         58345 non-null  float64
 3   Y                         58345 non-null  float64
 4   LONGITUDE                 58345 non-null  float64
 5   LATITUDE                  58345 non-null  float64
 6   DATE                      58345 non-null  object 
 7   TIME                      58345 non-null  object 
 8   ENVIRONMENT               58344 non-null  object 
 9   LIGHT                     58345 non-null  object 
 10  SURFACE_CONDITION         58345 non-null  object 
 11  TRAFFIC_CONTROL           58303 non-null  object 
 12  COLLISION_CLASSIFICATION  58345 non-null  object 
 13  IMPACT_TYPE               58345 non-null  object 
dtypes: flo

In [5]:
#check missing values
df_combine.isnull().sum()

COLLISION_ID                 0
LOCATION                     0
X                            0
Y                            0
LONGITUDE                    0
LATITUDE                     0
DATE                         0
TIME                         0
ENVIRONMENT                  1
LIGHT                        0
SURFACE_CONDITION            0
TRAFFIC_CONTROL             42
COLLISION_CLASSIFICATION     0
IMPACT_TYPE                  0
dtype: int64

In [6]:
# fill missing values
df_combine.loc[df_combine.ENVIRONMENT.isnull(), 'ENVIRONMENT'] = '00 - Unknown'
df_combine.loc[df_combine.TRAFFIC_CONTROL.isnull(),'TRAFFIC_CONTROL'] = '00 - Unknown'
df_combine.isnull().sum()

COLLISION_ID                0
LOCATION                    0
X                           0
Y                           0
LONGITUDE                   0
LATITUDE                    0
DATE                        0
TIME                        0
ENVIRONMENT                 0
LIGHT                       0
SURFACE_CONDITION           0
TRAFFIC_CONTROL             0
COLLISION_CLASSIFICATION    0
IMPACT_TYPE                 0
dtype: int64

## Match Nearest Neighbours to each accident

In [7]:
# file path of the ottawa neighbourhood_list.json
file_ottawa_neighbourhood = 'C:/Users/project file/neighborhood_list.json'
df_neighbour=pd.read_json(file_ottawa_neighbourhood)
# convert the dataframe into suitable form
list1 = [i for i in df_neighbour.columns]
list2 = [i for i in df_neighbour.index]
list3 = [i for i in df_neighbour[df_neighbour.index == 'lat'].values[0]]
list4 = [i for i in df_neighbour[df_neighbour.index == 'lng'].values[0]]

#create formal new formal dataframe for the neighbourhood
df_neighbourhood = pd.DataFrame({'Neighbourhoods':list1,'LATITUDE':list3, 'LONGITUDE':list4})

In [8]:
# we need to combine the dataframe to calculate the distance between the accidents and neighbourhoods.
def create_gdf(df, x='LONGITUDE', y='LATITUDE'):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[x],df[y]), crs={'init':'EPSG:4326'})

accidents_gdf = create_gdf(df_combine)
neighbourhood_gdf = create_gdf(df_neighbourhood)

# define the methods to calculate the nearst neighbourhoods:
def calculate_nearest(row, destination, val, col='geometry'):
    #create unary union
    dest_unary = destination['geometry'].unary_union
    #find closest point
    nearest_geom = nearest_points(row[col],dest_unary)
    #find the corresponding geom
    match_geom = destination.loc[destination.geometry==nearest_geom[1]]
    #get the corresponding value
    match_value = match_geom[val].to_numpy()[0]
    return match_value

#get the nearest geometry
accidents_gdf['nearest_geom'] = accidents_gdf.apply(calculate_nearest, destination=neighbourhood_gdf,val='geometry',axis=1)
#get the nearest neighbourhoodname:
accidents_gdf['Neighbourhoods'] = accidents_gdf.apply(calculate_nearest, destination=neighbourhood_gdf,val='Neighbourhoods',axis=1)

In [9]:
# we have assign neighbourhood to the accidents, now we would like to look at the groupby count:
accidents_gdf.groupby('Neighbourhoods').COLLISION_ID.count()

Neighbourhoods
Barrhaven                   3575
Cumberland                   587
Kanata                      5550
Nepean                      8229
Orleans                     5923
Ottawa Central              2551
Ottawa East                15084
Ottawa South                5031
Ottawa West Centre Town     2379
Rural Ottawa South          2171
Rural Ottawa West           4935
Stittsville                 2330
Name: COLLISION_ID, dtype: int64

## Match Nearest Weather Station
we need to do the weather analysis with the accidents, so we need to match each weather condition per hour with the accidents. And We need to find the nearest weather station for the accidents:

In [10]:
# filtering
station_file = 'C:/Users/project file/StationInventoryEN.csv'
station_df = pd.read_csv(station_file,skiprows=3)
station_df = station_df[['Name','Province','Station ID','Latitude (Decimal Degrees)','Longitude (Decimal Degrees)']]
station_df= station_df[station_df['Province']=='ONTARIO'].reset_index(drop=True)
station_df=station_df.rename(columns={'Latitude (Decimal Degrees)':'LATITUDE','Longitude (Decimal Degrees)':'LONGITUDE'})
station_df = station_df[station_df['Name'].apply(lambda x: 'OTTAWA' in x)].reset_index(drop=True)
station_list = [i for i in station_df.Name.values]

match the nearest weather based on the latitude and longitude with the traffic accident data

In [11]:
station_gdf = create_gdf(station_df)
#get the nearest Weather Station:
accidents_gdf['Weather_station'] = accidents_gdf.apply(calculate_nearest, destination=station_gdf,val='Name',axis=1)
accidents_gdf.Weather_station.unique()

array(['OTTAWA U OF O', 'OTTAWA ROCKCLIFFE A', 'OTTAWA CITY HALL',
       'OTTAWA HAZELDEAN', 'OTTAWA BRITANNIA', 'OTTAWA',
       'OTTAWA ALBION RD', 'OTTAWA BILLINGS BRIDGE', 'OTTAWA NRC',
       "OTTAWA MACDONALD-CARTIER INT'L A", 'OTTAWA BECKWITH RD',
       'OTTAWA RIDEAU WARD', 'OTTAWA CDA', 'OTTAWA HOGS BACK',
       'OTTAWA ALTA VISTA', 'OTTAWA KANATA', 'OTTAWA SOUTH MARCH',
       'OTTAWA STOLPORT A', 'OTTAWA LEMIEUX ISLAND'], dtype=object)

In [31]:
accidents_df = accidents_gdf.drop(columns = ['X','Y','geometry','nearest_geom'])

## Filtering Weather Data Based on the nearest Weather Station 

In [15]:
filelist=['ontario_1_1','ontario_1_2','ontario_2_1-004','Ontario_2_2','Ontario_4-006']
yearlist=[2013,2014,2015,2016,2017]
root_dir = 'C:/Users/project file'
weather_columns = ['X.Date.Time',
 'Year',
 'Month',
 'Day',
 'Time',
 'Temp...C.',
 'Temp.Flag',
 'Dew.Point.Temp...C.',
 'Dew.Point.Temp.Flag',
 'Rel.Hum....',
 'Rel.Hum.Flag',
 'Wind.Dir..10s.deg.',
 'Wind.Dir.Flag',
 'Wind.Spd..km.h.',
 'Wind.Spd.Flag',
 'Visibility..km.',
 'Visibility.Flag',
 'Stn.Press..kPa.',
 'Stn.Press.Flag',
 'Hmdx',
 'Hmdx.Flag',
 'Wind.Chill',
 'Wind.Chill.Flag',
 'Weather.',
 'X.U.FEFF..Station.Name.',
 'X.Province.']

In [ ]:
def getweather_ottawa():
    df_weather=pd.DataFrame()
    z = 0
    entries = 0
    for file in filelist:
        print('reading '+str(file))
        dtype = dict((i,convert_dtype) for i in weather_columns)
        station_file = pd.read_csv('{}/Ontario/{}.csv'.format(root_dir,file), chunksize=1000000, low_memory=False)
        #station_file['Year'] = station_file['Year'].astype('int')
        #station_file['X.U.FEFF..Station.Name.'].astype('str')
        for chunk in station_file:
            print('processing...'+str(z))
            y = chunk[(chunk['X.U.FEFF..Station.Name.'].isin(station_list))]
            x = y[y['Year'].isin(yearlist)]
            #set_index
            x_size=x.size
            #index=list(range(entries,x_size+entries))
            x = x.reset_index(drop=True)
            x['Weather_ID'] = x.index+entries
            if x_size > 0:
                #x.to_csv(root_dir+'/Chunk'+str(z)+'_weather.csv', encoding='utf-8', index=False)
                print('Finish...'+str(z))
                z+=1
                entries += x_size
                df_weather=df_weather.append(x,ignore_index=True)
    return df_weather.to_csv(root_dir+'/ottawa_weather.csv', encoding='utf-8', index=False)

In [17]:
df_weather = pd.read_csv(root_dir+'/ottawa_weather.csv')

C:\Users\gongh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,8,10,12,14,18,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051776 entries, 0 to 1051775
Data columns (total 27 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   X.Date.Time              1051776 non-null  object 
 1   Year                     1051776 non-null  int64  
 2   Month                    1051776 non-null  int64  
 3   Day                      1051776 non-null  int64  
 4   Time                     1051776 non-null  object 
 5   Temp...C.                86735 non-null    float64
 6   Temp.Flag                39 non-null       object 
 7   Dew.Point.Temp...C.      86737 non-null    float64
 8   Dew.Point.Temp.Flag      8 non-null        object 
 9   Rel.Hum....              86721 non-null    float64
 10  Rel.Hum.Flag             53 non-null       object 
 11  Wind.Dir..10s.deg.       85730 non-null    float64
 12  Wind.Dir.Flag            167 non-null      object 
 13  Wind.Spd..km.h.          86739 non-null   

The Data format is not consitent based on the time format, we need to split the date to year,month,day based on the business requirements.

In [13]:
from datetime import datetime
# convert 12 timecounting to 24 timecounting
def timeconvert(str1):
    if str1[-2:] == "AM" and str1[:2] == "12":
            return "00" + str1[2:-6]
    elif str1[-2:] == "AM" and (str1[:2] =='11'or str1[:2]=='10'):
            return str1[:-6]
    elif str1[-2:] == "AM":
            str1 = '0' + str1
            return str1[:5]
    elif str1[-2:] == "PM" and str1[:2] == "12":
            return str1[:5]
    elif str1[-2:] == "PM" and (str1[:2] =='11'or str1[:2]=='10'):
            return str(int(str1[:2]) + 12) +str1[2:5]
    else:
            str1 = '0' + str1
            return str(int(str1[:2]) + 12) + str1[2:5]
    
# convert datetime
def str_to_datetime(x):
    try:
         return datetime.strptime(x,'%Y-%m-%d')
    except:
         return datetime.strptime(x,'%m/%d/%Y')

# convert rounded hour
def rounded_time(x):
    global str
    global int
    if x[:2]=='23':
        x = '23:00'
        return x
    else:
        if int(x[-2:]) >=45 and x[0]=='0' and x[1]!='9':
            return "0" + str(int(x[:2])+1)+':00'
        elif int(x[-2:]) >=45:
            return str(int(x[:2])+1)+':00'
        else:
            return x[:3]+'00'
            
            
## get date and time columns
def get_timecolumns(df):
    global str
    global int
    # str_to date
    df['DATE'] = df['DATE'].apply(lambda x: str_to_datetime(x))
    # create time columns
    df['YEAR'] = df['DATE'].apply(lambda x : x.year)
    df['MONTH'] = df['DATE'].apply(lambda x : x.month)
    df['DAY'] = df['DATE'].apply(lambda x: x.day)
    df['TIME'] = df['TIME'].apply(lambda x : timeconvert(x))
    df['ROUNDED_TIME'] = df['TIME'].apply(lambda x:rounded_time(x))
    return df

In [32]:
accidents_df['DATE']=accidents_df['DATE'].apply(lambda x:str(x)[:10])

In [33]:
new_df = get_timecolumns(accidents_df)

In [ ]:
weather_ID = [i for i in df_weather.index.values]
df_weather['WEATHER_ID']=weather_ID
df_weather = df_weather.drop(columns=['Weather_ID'])

In [38]:
df_weather = df_weather.rename(columns={'Year':'YEAR','Month':'MONTH','Day':'DAY','Time':'ROUNDED_TIME','X.U.FEFF..Station.Name.':'Weather_station'})
df_merged = pd.merge(new_df,df_weather, on=['YEAR','MONTH','DAY','ROUNDED_TIME','Weather_station'], how='left')
df_merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 58345 entries, 0 to 58344
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   COLLISION_ID              58345 non-null  object        
 1   LOCATION                  58345 non-null  object        
 2   LONGITUDE                 58345 non-null  float64       
 3   LATITUDE                  58345 non-null  float64       
 4   DATE                      58345 non-null  datetime64[ns]
 5   TIME                      58345 non-null  object        
 6   ENVIRONMENT               58345 non-null  object        
 7   LIGHT                     58345 non-null  object        
 8   SURFACE_CONDITION         58345 non-null  object        
 9   TRAFFIC_CONTROL           58345 non-null  object        
 10  COLLISION_CLASSIFICATION  58345 non-null  object        
 11  IMPACT_TYPE               58345 non-null  object        
 12  Neighbourh

In [ ]:
df_merged.to_csv(root_dir+'/master_data.csv',index=False)